In [25]:
import datetime
import pandas as pd
import numpy as np
from requests import get
from bs4 import BeautifulSoup


URl_MAIN = 'https://www.nsi.bg'

# Bulgarian National Statistics Institute
BG_NSI_URL = {
    'main': 'https://www.nsi.bg',
    'pages': {
        'life_expectancy':
            '/bg/content/3018/смъртност-и-средна-продължителност-на-предстоящия-живот-на-населението-по-местоживеене',
            'mortality_per_week': '/bg/node/18121/',
    },
    'files': {
        'mortality_per_week': '/bg/node/18121/',
        'life_expectancy': '/sites/default/files/files/data/timeseries/Pop_3.1_tab_mortality_DR.xls',
    }
}


In [32]:
url = BG_NSI_URL['main'] + BG_NSI_URL['pages']['mortality_per_week']

req = get(url)
page = BeautifulSoup(req.content, 'lxml')
mortality_per_week_file = page.find(lambda tag: tag.name == 'a' and 'Умирания в България по седмици в периода' in tag.text)
mortality_per_week_path = mortality_per_week_file['href']
file_req = get(BG_NSI_URL['main']+mortality_per_week_path)
date = datetime.date.today()

df = pd.read_excel(file_req.content, engine='openpyxl')

df.dropna(axis=1, how='all', inplace=True)
headers = df.iloc[2]
df.dropna(axis=0, how='any', inplace=True)
df.columns = headers
df.rename(columns={ df.columns[0]: "Year" },inplace=True)
df.replace('x', np.nan, inplace = True)

avg_years_start = df[df['Year'].str.startswith('2015')].index.values[0]
avg_years_end = df[df['Year'].str.startswith('2019')].index.values[0]
analized_year = df[df['Year'].str.startswith('2020')].index.values[0]

excess_death = pd.DataFrame()
excess_death['average_death'] = df.loc[avg_years_start:avg_years_end].mean()
death_2020 = df.loc[analized_year]
excess_death['death_2020'] = death_2020
excess_death = excess_death[excess_death['death_2020']!='-']
excess_death['excess_death'] = excess_death['death_2020'] - excess_death['average_death']

excess_death.reset_index(inplace=True)
excess_death.rename(columns={  excess_death.columns[0]: "Week" },inplace=True)
excess_death['Week'] = excess_death['Week'].str.replace('W', '')

In [33]:
excess_death['Week'] = pd.to_numeric(excess_death['Week'])

In [41]:
sum(excess_death[(excess_death['Week']>=0)& (excess_death['Week']<=100)]['excess_death'])

15110.199999999997